# Development Notebook

#### CCD Implementation

In [2]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire
from fibsem.structures import BeamType

import numpy as np
import matplotlib.pyplot as plt



microscope, settings = utils.setup_session(ip_address="10.0.0.1", manufacturer="Thermo")


c:\Users\Admin\.conda\envs\liftout\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Admin\.conda\envs\liftout\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Admin\.conda\envs\liftout\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


2023-09-06 17:28:23,125 — root — INFO — connect_to_microscope:453 — Microscope client connecting to [10.0.0.1:7520]
Client connecting to [10.0.0.1:7520]...
Client connected to [10.0.0.1:7520]
2023-09-06 17:28:23,129 — root — INFO — connect_to_microscope:455 — Microscope client connected to [10.0.0.1:7520]
2023-09-06 17:28:23,132 — root — INFO — connect_to_microscope:459 — Microscope client connected to model Helios G4 PFIB UNSPECIFIED with serial number 9925949 and software version 14.5.1.432.
2023-09-06 17:28:23,141 — root — INFO — setup_session:206 — Finished setup for session: demo_2023-09-06-05-28-23PM


In [3]:

import napari
viewer = napari.Viewer()



2023-09-06 17:28:25,073 — numexpr.utils — INFO — _init_num_threads:160 — NumExpr defaulting to 4 threads.


In [4]:
from pprint import pprint 

microscope.connection.imaging.set_active_view(4)
microscope.connection.imaging.set_active_device(3) #CCD = DEVICE 3, BUT VIEW=4 USUALLY WHAT?

import time
from fibsem.structures import FibsemImage


from napari.qt.threading import thread_worker


@thread_worker
def _acquire_image(microscope):
    i = 0
    images = []
    while True:

        print(f"Acquiring image... {i:04d}")
        image = microscope.connection.imaging.get_image()
        image = FibsemImage(image.data, None)
        # images.append(image)

        yield image
        
        time.sleep(0.2)
        i += 1
        if i > 100:
            break




def _on_yield(image):
    try:
        viewer.layers["data"].data = image.data
    except KeyError:
        viewer.add_image(image.data, name="data")


worker = _acquire_image(microscope)
worker.yielded.connect(_on_yield)
worker.finished.connect(lambda: print("Finished!"))
worker.finished.connect(viewer.close)
worker.start()



Acquiring image... 0000


Acquiring image... 0001
Acquiring image... 0002
Acquiring image... 0003
Acquiring image... 0004
Acquiring image... 0005
Acquiring image... 0006
Acquiring image... 0007
Acquiring image... 0008
Acquiring image... 0009
Acquiring image... 0010
Acquiring image... 0011
Acquiring image... 0012
Acquiring image... 0013
Acquiring image... 0014
Acquiring image... 0015
Acquiring image... 0016
Acquiring image... 0017
Acquiring image... 0018
Acquiring image... 0019
Acquiring image... 0020
Acquiring image... 0021
Acquiring image... 0022
Acquiring image... 0023
Acquiring image... 0024
Acquiring image... 0025
Acquiring image... 0026
Acquiring image... 0027
Acquiring image... 0028
Acquiring image... 0029
Acquiring image... 0030
Acquiring image... 0031
Acquiring image... 0032
Acquiring image... 0033
Acquiring image... 0034
Acquiring image... 0035
Acquiring image... 0036
Acquiring image... 0037
Acquiring image... 0038
Acquiring image... 0039
Acquiring image... 0040
Acquiring image... 0041
Acquiring image

In [ ]:

# plot all images

fig, axes = plt.subplots(1, len(images), figsize=(20, 20))
for i, image in enumerate(images):
    axes[i].imshow(image.data, cmap='gray')
    axes[i].set_title(f'Image {i}')
    axes[i].axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.show()